# NP-Hard Problems

The purpose of this assignment is to familiarize yourself with different approaches to solving NP-hard problems in practice, especially via integer programming.

As in previous assignments, we will use [NetworkX](https://networkx.github.io/) to manipulate graphs and [PuLP](http://pythonhosted.org/PuLP/) to solve linear and integer programs.

In [1]:
import networkx as nx
import pulp

We will use two graph datasets for this assignment, both available in [GML](https://en.wikipedia.org/wiki/Graph_Modelling_Language) format. The first is a well-known graph of the social network of a karate club [1]. The second is a network representing the western states power grid in the US [2].

In [2]:
karate = nx.read_gml('karate.gml',label='id')
power = nx.read_gml('power.gml',label='id')

[1] W. Zachary, An information flow model for conflict and fission in small groups, Journal of Anthropological Research 33, 452-473 (1977). 

[2] D. J. Watts and S. H. Strogatz, Collective dynamics of 'small world' networks, Nature 393, 440-442 (1998). 

## Independent Set

Recall that an *independent set* in a graph is a set of nodes such that no two nodes are adjacent (connected by an edge). Finding the maximum size of an independent set in a graph is an NP-hard problem, so there is no known polynomial-time algorithm to solve it.

### Problem 1

Provide an *integer programming* formulation for this problem here. The graph is denoted by $G = (V, E)$. Use a binary variable $x_i \in \{0,1\}$ for each node $i \in V$ (you do not need any other variables).

#### Solution:

\begin{equation}
G = n \; nodes, m \; edges \\
x_i = 1 \; if \; node_i \in IS, 0 \; o.w. \\
max \sum_{i=1}^{n} x_i \; s.t. \\ 
x_i + x_j \leq 1 \; \; \forall \ (v_i,v_j) \in E, \; i < j \\
x_i \in \{0,1\} \; \forall \; 1 \leq i \leq n
\end{equation}

### Problem 2

Implement a function that solves the integer program given a graph as input.

In [3]:
def independent_set_ip(graph):
    """Computes a maximum independent set of a graph using an integer program.
    
    Args:
      - graph (nx.Graph): an undirected graph
    
    Returns:
        (list[(int, int)]) The IP solution as a list of node-value pairs.
        
    """    
    xi=[]
    for i in graph.nodes():
        xi.append(i)
        
    problem = pulp.LpProblem("Max IS IP Problem", pulp.LpMaximize)
    x = pulp.LpVariable.dict('x', [str(j) for j in xi], lowBound=0, cat='Integer')
        
    for i in xi:
        problem += x[str(i)] <= 1
        
    for j in graph.edges(data=True):
        problem += x[str(j[0])]+x[str(j[1])] <= 1
    
    problem += pulp.lpSum([x[str(i)] for i in xi])
    problem.solve()
    
    list = []
    for k in xi:
        y = x[str(k)].value()
        z = [(int(k), int(y))]
        list.extend(z)
        
    return list

The following code outputs the size of the sets computed by your function.

In [4]:
def set_weight(solution):
    """Computes the total weight of the solution of an LP or IP for independent set.
    
    Args:
      - solution (list[int, float]): the LP or IP solution
    
    Returns:
      (float) Total weight of the solution
    
    """
    return sum(value for (node, value) in solution)

In [5]:
karate_ind_set = independent_set_ip(karate)
print("Size of karate set = {}".format(set_weight(karate_ind_set)))
power_ind_set = independent_set_ip(power)
print("Size of power set = {}".format(set_weight(power_ind_set)))

Size of karate set = 20
Size of power set = 2738


### Problem 3

Take the *linear programming relaxation* of your integer program and implement a function to solve it. This simply means that in your integer program, you should replace each constraint $x_i \in \{0,1\}$ with $0 \leq x_i \leq 1$.

In [6]:
def independent_set_lp(graph):
    """Computes the solution to the linear programming relaxation for the
    maximum independent set problem.
    
    Args:
      - graph (nx.Graph): an undirected graph
    
    Returns:
        (list[(int, float)]) The LP solution as a list of node-value pairs.
        
    """    
    xi=[]
    for i in graph.nodes():
        xi.append(i)
        
    problem = pulp.LpProblem("Max IS IP Problem", pulp.LpMaximize)
    x = pulp.LpVariable.dict('x', [str(j) for j in xi], lowBound=0, cat='Continuous')
        
    for i in xi:
        problem += x[str(i)] <= 1
        
    for j in graph.edges(data=True):
        problem += x[str(j[0])]+x[str(j[1])] <= 1
    
    problem += pulp.lpSum([x[str(i)] for i in xi])
    problem.solve()
    
    list = []
    for k in xi:
        y = x[str(k)].value()
        z = [(int(k), float(y))]
        list.extend(z)
        
    return list

Let's see how the LP solutions compares to those of the IPs.

In [7]:
karate_ind_set_relax = independent_set_lp(karate)
print("Value of karate set = {}".format(set_weight(karate_ind_set_relax)))
power_ind_set_relax = independent_set_lp(power)
print("Value of power set = {}".format(set_weight(power_ind_set_relax)))

Value of karate set = 20.5
Value of power set = 2757.9999999999786


A heuristic way to convert a fractional solution to an independent set is as follows. For each node $i$, include the node in the set if $x_i > 1/2$, and discard it otherwise. This will yield a set of $\alpha$ nodes which have $\beta$ edges between them. By removing at most one node for each edge, this yields an independent set of size at least $\alpha - \beta$.

Implement this rounding procedure.

In [8]:
def round_solution(solution, graph):
    """Finds the subgraph corresponding to the rounding of
    a solution to the independent set LP relaxation.
    
    Args:
      - solution (list[(int, float)]): LP solution
      - graph (nx.Graph): the original graph
      
    Returns:
        (nx.Graph) The subgraph corresponding to rounded solution
    
    """
    G_prime = nx.Graph()
    for edge in graph.edges():
        G_prime.add_edge(edge[0], edge[1])
    
    for i in solution:
        if i[1] <= 1./2:
            G_prime.remove_node(i[0])
    return G_prime

The following function assesses the quality of the heuristic approach.

In [9]:
def solution_quality(rounded, optimal):
    """Computes the percent optimality of the rounded solution.
    
    Args:
      - rounded (nx.Graph): the graph obtained from rounded LP solution
      - optimal: size of maximum independent set

    """
    num_nodes = rounded.number_of_nodes() - rounded.number_of_edges()
    return float(num_nodes) / optimal

Let's check the quality of this approach compared to the optimal IP solutions.

In [10]:
karate_rounded = round_solution(karate_ind_set_relax, karate)
karate_quality = solution_quality(karate_rounded, set_weight(karate_ind_set))
print("Quality of karate rounded solution = {:.0f}%".format(karate_quality*100))

power_rounded = round_solution(power_ind_set_relax, power)
power_quality = solution_quality(power_rounded, set_weight(power_ind_set))
print("Quality of power rounded solution = {:.0f}%".format(power_quality*100))

Quality of karate rounded solution = 90%
Quality of power rounded solution = 95%
